# Necesario tener importar las siguientes librerias

In [ ]:
import json
import tweepy
# General:
import tweepy           # Para consumir la API de Tweeter
import pandas as pd     # Para análisis de datos
import numpy as np      # Para cálculo numérico

import networkx as nx
import itertools
import matplotlib.pyplot as plt
from pyvis.network import Network

import pickle
import pandas as pd 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

import os
import sys
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Embedding, LSTM ,GRU
from keras.layers.embeddings import  Embedding
from keras.models import Sequential
from keras.initializers import Constant
from sklearn.model_selection import train_test_split

##  claves de autenticacion de twitter

In [ ]:
consumer_key='CTXchN99VNe7jl8MgZrE47AIm'
consumer_secret= 'faQ0b0r6FyfblBIPfJe0k6qjlEEqbynVmghjaKuzZhjskggjFq'
access_token = '1118494816105574401-JTGPAQhrSsvFSrd9O0eN03HkFVYjAK'
access_secret = 'OWg6aPlumQPSlXFn2kAiQcByNFkWyse0nE9JgXxQlglnR'

#  funcion de llamada a la api

In [ ]:
def twitter_setup():
    """
    Función de utilidad para configurar la API de Twitter
    con nuestras claves de acceso.
    """
    # Autenticación y acceso usando claves:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    # Retornar API con autenticación:
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api

# extraccion de tweets

In [ ]:
def tweet_caller_csv(id_):
        extractor = twitter_setup() 
        u = extractor.get_user(id_)
        private= u.protected

        if private == False:
            try: 
                    
                    name = u.screen_name  
                    tweets = extractor.user_timeline(id=id_, count=200,tweet_mode= 'extended')

            except:
                    print('Error in try 01: caller')

            try:    #CREATE A DATAFRAME   ##02
                    alltweet = []
                    alltweet.extend(tweets)
                    oldest = alltweet[-1].id - 1
                    try:
                        while len(tweets) > 0:
                                tweets = extractor.user_timeline(id= id_,count=200,tweet_mode= 'extended',max_id=oldest)
                                alltweet.extend(tweets)
                                oldest = alltweet[-1].id - 1
                    except ConnectionError:
                        print('ConnectionError')
                    result = [[tweet.id, tweet.created_at, tweet.full_text, tweet.entities] for tweet in alltweet]
                    data= pd.DataFrame(result, columns=['id','date','text', 'entities'])

            except:
                    print('Error in try: 02, dataframe')

            try:    #Extract hashtags     ##03
                    hashtags=[]
                    for key in data['entities']:
                        try:
                            hashtags.append(key['hashtags'][0]['text'])
                        except: 
                            hashtags.append('null')
                            pass
            except: 
                    print('Error in try: 03, hashtags')


            try:    #Extract hashtags     ##04
                    menciones = []
                    menciones_id = []
                    for key in data['entities']:
                        try:
                            menciones.append(key['user_mentions'][0]['name'])
                            menciones_id.append(key['user_mentions'][0]['id'])
                        except: 
                            menciones.append('null')
                            menciones_id.append('null')
                            pass
            except: 
                    print('Error in try: 04, entities')   


            try:    #Extract hashtags     ##05

                    retweet_count = [tweet.retweet_count for tweet in alltweet]
            except: 
                    print('Error in try: 05, retweet_count')


            try:    #Extract hashtags     ##06

                    like = [tweet.favorite_count for tweet in alltweet]
            except: 
                    print('Error in try: 06, likes')



            try:    #unique data          ##07
                    hashtags= pd.DataFrame(hashtags, columns= ['hashtags'])
                    menciones = pd.DataFrame(menciones, columns= ['menciones'])
                    menciones_id = pd.DataFrame(menciones_id, columns= ['menciones_id'])
                    menciones_combinado = pd.concat([menciones, menciones_id], axis= 1)
                    retweet_count = pd.DataFrame(retweet_count, columns= ['numero_retweet'])
                    like_count = pd.DataFrame(like, columns=['numero_likes'])

            except:
                    print('Error in try: 07, dataframes')

            try:    #unique data          ##08
                    data=pd.concat([data,hashtags, menciones_combinado, retweet_count, like_count], axis= 1)

            except:
                    print('Error in try: 08, concat')



            try:    #drop columna and RT  ##09
                data= data.drop(columns='entities')
                data = data.drop(data[data.text.str.contains("RT")].index)

            except:
                    print('Error in try: 09, drop') 

            data.to_csv(name+'.csv', line_terminator='\rn')
        else:
            print('private user')
        return 

# Extraccion de followers

In [ ]:
def extractor_follows(id_):
    extractor= twitter_setup()
    ids = []
    u = extractor.get_user(id_)
    name = u.screen_name
    try:
        for page in tweepy.Cursor(extractor.followers_ids, id=id_).pages():
            ids.extend(page)
    except: ConnectionError      
    ids=pd.DataFrame(ids,columns=[name],dtype=object)
    return ids 

In [ ]:
def extractor_follows_01(id_):
    extractor= twitter_setup()
    ids = []
    u = extractor.get_user(id_)
    name = u.screen_name
    try:
        for page in tweepy.Cursor(extractor.followers_ids, id=id_).pages():
            ids.extend(page)
    except: ConnectionError      
    
    return ids, name 

In [ ]:
def grid_of_followers(id_):
    
        extractor= twitter_setup()
        
        df = extractor_follows(id_)
        u = extractor.get_user(id_)
        private= u.protected
        
        if private== False:
            name_user = u.screen_name
            
            
            try:
                followers=[]
                names= []
                for i in df[name_user]:
                    u = extractor.get_user(i)
                    private= u.protected
                    
                    if private == False:
                    
                        follower, name= extractor_follows_01(i)
                        followers.append(follower)
                        names.append(i)
                                      
                    else:
                        print('error')
                        pass
                        
                followers_dt= pd.DataFrame(followers,dtype=object).transpose()
                names_series= pd.Series(names)
                followers_dt=followers_dt.rename(columns=names_series)
                grid= pd.concat([df,followers_dt],axis=1)
                grid.to_csv(name_user +'grid_of_follows.csv',mode= 'a')
            except:
                print('error 2')
                pass
            
        else:
            print('private user')
            pass
        return 

# Funcion prediccion de sentimiento 

In [ ]:
def prediciones_keras(tweets):
    keras_tweet = pickle.load(open('model_predict_keras.sav', 'rb'))
    dt= pd.read_csv('train_tweet.csv')
    sentences= dt['content'].values
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentences)
    word_index= tokenizer.word_index
    
    tweet_01=[]
    for i in list(tweets):
        try:
            lista= []
            a=i.split()
            a= list(a)
            for v in a:
                lista.append(v.encode('ascii', 'ignore').decode('ascii'))
            tweet_01.append(lista)
        except:
            continue
            
    tweet_02=[]
    for i in tweet_01:
        lista=[]
        for line in i:
            tokens = word_tokenize(line)
            tokens = [w.lower() for w in tokens]
            table = str.maketrans('','',string.punctuation)
            stripped = [w.translate(table) for w in tokens]
            words= [word for word in stripped if word.isalpha()]
            stop_words = set(stopwords.words('spanish'))
            words= [w for w in words if not w in stop_words]
            lista.append(words)
        tweet_02.append(lista)  
        
    predictions_mean=[]
    for i in tweet_02:
        try:
            predictions= tokenizer.texts_to_sequences(i)
            predictions= pad_sequences(predictions, padding= 'post', maxlen= 30)
            predictions_mean.append(keras_tweet.predict(predictions).mean())
        except:
            pass
    global_predicition= sum(predictions_mean)/len(predictions_mean)
    return global_predicition

# Construccion del Grafico 

In [ ]:
def unique(list1): 
  
    
    unique_list = [] 
        
    for x in list1: 
        
        if x not in unique_list and x!= 0: 
            unique_list.append(x) 
    
    return unique_list

In [ ]:
def graph_network(dt):
    dt = dt.drop(columns='Unnamed: 0')
    dt = dt.fillna(0)
    name= dt.columns[0]
    df_list = []
    for col in dt:
        for v in dt[col]:
            df_list.append(v)
            
    nodes_02 = unique(df_list)
    
    edges_02= []
    for col in dt:
        for v in dt[col]:
            if v != 0:
                edges_02.append([col,v])
    edges_03=[]
    for i in edges_02:
        edges_03.append(tuple(i))
    
    g= Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    g.add_node(name,label=name,color='red',)

    for i in nodes_02:
        g.add_node(i, label= i)

    g.add_edges(edges_03)
    g.show(name+'grahp.html')

In [ ]:
def sentiment_graph_network(dt):
    dt = dt.drop(columns='Unnamed: 0')
    dt = dt.fillna(0)
    name= dt.columns[0]
    df_list = []
    for col in dt:
        for v in dt[col]:
            df_list.append(v)
            
    nodes_02 = unique(df_list)
    
    edges_02= []
    for col in dt:
        for v in dt[col]:
            if v != 0:
                edges_02.append([col,v])
    edges_03=[]
    for i in edges_02:
        edges_03.append(tuple(i))
    
    dicts= {}
    for i in nodes_02:
        extractor= twitter_setup()
        if  i== name:
            try:
                    u=extractor.get_user(name)
                    id_= u.id
                    tweet_caller_csv(id_)
                    tweets = pd.read_csv(name+'.csv', dtype=object)
                    tweets= tweets['text']
                    dicts[i]= prediciones_keras(tweets)
            except:
                pass
        else:
            
            try:
                u = extractor.get_user(i)
                name = u.screen_name
                tweet_caller_csv(i)
                tweets = pd.read_csv(name+'.csv', dtype=object)
                tweets= tweets['text']
                dicts[i]= prediciones_keras(tweets)
                
            except:
                pass
        
    for i in dicts:
        if dicts.get(i)>0.25:
            dicts[i]= 'red'
        else:
            dicts[i]= 'blue'  
        
    g= Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
    g.add_node(name,label=name,color='green')

    for i in dicts:
        g.add_node(i,lable= i,color=dicts.get(i))

    g.add_edges(edges_03)
    g.show(name+'grahp.html')

In [ ]:
"""%%time
tweet_caller_csv(924114887722364929)"""


In [ ]:
"""%%time
grid_of_followers(924114887722364929)
Wall time: 9.45 s"""

In [ ]:
#dt = pd.read_csv('Carlos0520683grid_of_follows.csv',dtype=object)


In [ ]:
#graph_network(dt)

In [ ]:
'''tweets = pd.read_csv('Carlos0520683.csv', dtype=object)
tweets= tweets['text']

prediciones_keras(tweets)
output= 0.3338531681469509'''

In [ ]:
%%time
dt = pd.read_csv('Carlos0520683grid_of_follows.csv',dtype=object)
sentiment_graph_network(dt)